# Student Queries 

In [1]:
from data201 import db_connection, df_query
from pandas import DataFrame
conn = db_connection('sheql.ini')
cursor = conn.cursor()

### Dataframe visualization for testing purposes

In [2]:
def display_results(cursor):
    """
    If there are results in the cursor from a call to a
    stored procedure, display the results in a dataframe. 
    """
    columns = result.description
    
    if columns == None:
        return 
    
    else:
        column_names = [column_info[0] 
                        for column_info in columns]

        rows = cursor.fetchall()
    
        if len(rows) > 0:
            df = DataFrame(rows)
            df.columns = column_names
            
            display(df)
        
        else:
            return

## Get student course schedule

In [3]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_course_schedule')

cursor.execute(
    """
    CREATE PROCEDURE get_student_course_schedule(
        IN sUserName VARCHAR(50)
    )
    BEGIN
        SELECT 
            c.course_id AS "Course ID",
            c.name AS "Course Name",
            CONCAT(takes.start_time, "-", takes.end_time) AS "Class Times"
        FROM users u
        JOIN student s USING (user_id)
        JOIN takes USING (student_id)
        JOIN course c USING (course_id)
        JOIN teaches USING (course_id)
        JOIN teacher t USING (teacher_id)
        WHERE u.username = sUserName
            AND teaches.course_id = takes.course_id
            AND teaches.grade_level = s.grade_level
        GROUP BY c.course_id,
            c.name,
            s.grade_level,
            takes.start_time,
            takes.end_time
        ORDER BY takes.start_time, c.course_id;
    END
    """
)

In [4]:
sUserName = 'stu_jhays64'

cursor.callproc('get_student_course_schedule', (sUserName,))

for result in cursor.stored_results():
    display_results(result)

,Course ID,Course Name,Class Times
0,8,Computers,08:30:00-09:15:00
1,6,PE,09:25:00-10:10:00
2,5,Art,10:20:00-11:05:00
3,4,History,11:15:00-12:00:00
4,2,Science,13:00:00-13:50:00
5,7,Music,14:00:00-14:45:00


## Get individual student data (name, DOB, Grade Level) 

In [5]:
cursor.execute('DROP PROCEDURE IF EXISTS get_individual_student')

cursor.execute(
    """
    CREATE PROCEDURE get_individual_student(
        IN sUserName VARCHAR(50)
    )
    BEGIN
        SELECT 
        	CONCAT(s.first_name, ' ', s.last_name) AS "Student Name", 
            s.date_of_birth AS "DOB", 
            s.grade_level AS "Grade Level"
        FROM student s
        JOIN users u USING (user_id) 
        WHERE u.username = sUserName;
    END
    """
)

In [6]:
sUserName = 'stu_jhays64'

cursor.callproc('get_individual_student', (sUserName,))

for result in cursor.stored_results():
    display_results(result)

,Student Name,DOB,Grade Level
0,Joshua Hays,2010-12-26,10


## Display Student Courses 

In [7]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_courses')

cursor.execute(
    """
    CREATE PROCEDURE get_student_courses(
        IN sUserName VARCHAR(50)
    )
    BEGIN
        SELECT DISTINCT  c.name AS "Subject"
        FROM users u
        JOIN student s ON u.user_id = s.user_id
        JOIN takes t ON s.student_id = t.student_id
        JOIN course c ON t.course_id = c.course_id
        WHERE u.username = sUserName;
    END
    """
)

In [8]:
sUserName = 'stu_jhays64'

cursor.callproc('get_student_courses', (sUserName,))

for result in cursor.stored_results():
    display_results(result)

,Subject
0,Science
1,History
2,Art
3,PE
4,Music
5,Computers


# Get Guardian Info (name and phone number) 

In [9]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_guardian')

cursor.execute(
    """
    CREATE PROCEDURE get_student_guardian(
        IN sUserName VARCHAR(50)
    )
    BEGIN
        SELECT CONCAT(g.first_name, ' ', g.last_name) AS "Name", 
            g.phone_number AS "Phone Number" 
        FROM users u
            JOIN student s ON u.user_id = s.user_id
            JOIN guardian_student_relationship gs ON gs.student_id = s.student_id
            JOIN guardian g ON g.guardian_id = gs.guardian_id
        WHERE u.username = sUserName;
    END
    """
)

In [10]:
sUserName = 'stu_jhays64'

cursor.callproc('get_student_guardian', (sUserName,))

for result in cursor.stored_results():
    display_results(result)

,Name,Phone Number
0,Earl Hays,(843) 880-4436
1,Kenneth Stokes,(819) 622-6979


# Get Student's Homeroom Teacher Name and Email 

In [11]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_hometeacher')

cursor.execute(
    """
    CREATE PROCEDURE get_student_hometeacher(
        IN sUserName VARCHAR(50)
    )
    BEGIN
        SELECT 
            CONCAT(t.first_name,' ', t.last_name) as "Teacher Name",
            u_t.email AS "Teacher Email"
        FROM users u_s
        JOIN student s ON u_s.user_id = s.user_id
        JOIN teacher t ON s.homeroom_teacher_id = t.teacher_id
        JOIN users u_t ON t.user_id = u_t.user_id
        WHERE u_s.username = sUserName;
    END
    """
)

In [12]:
sUserName = 'stu_jhays64'

cursor.callproc('get_student_hometeacher', (sUserName,))

for result in cursor.stored_results():
    display_results(result)

,Teacher Name,Teacher Email
0,Cheyenne Ball,tea_chey.ball@moreno.info


# Get Student's Grades 

In [13]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_grades')

cursor.execute(
    """
    CREATE PROCEDURE get_student_grades(
        IN sUserName VARCHAR(50)
    )
    BEGIN
            WITH student_grades AS (
              SELECT 
                  c.name AS course_name,
                  ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) AS weighted_average
              FROM users u
              JOIN student s ON u.user_id = s.user_id
              JOIN grade_details gd ON s.student_id = gd.student_id
              JOIN course c ON gd.course_id = c.course_id
              WHERE u.username = sUserName
              GROUP BY c.course_id, c.name
        )
        SELECT 
            course_name AS "Course Name",
            weighted_average AS "Weighted Average",
            CASE 
                WHEN weighted_average >= 90 THEN 'A'
                WHEN weighted_average >= 80 THEN 'B'
                WHEN weighted_average >= 70 THEN 'C'
                WHEN weighted_average >= 60 THEN 'D'
                ELSE 'F'
            END AS "Letter Grade"
        FROM student_grades;
    END
    """
)

In [14]:
sUserName = 'stu_jhays64'

cursor.callproc('get_student_grades', (sUserName,))

for result in cursor.stored_results():
    display_results(result)

,Course Name,Weighted Average,Letter Grade
0,Science,74.40,C
1,History,94.45,A
2,Art,86.05,B
3,PE,78.65,C
4,Music,73.65,C
5,Computers,74.65,C


## Get student's assignment grades

In [15]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_assignment_grades')

cursor.execute(
    """
    CREATE PROCEDURE get_student_assignment_grades(
        IN sUserName VARCHAR(50),
        IN cid INT
    )
    BEGIN
            WITH student_grades AS (
              SELECT 
                  gd.grade_type as assignment,  
                  gd.score as score,
                  gd.weight as weight,
                  ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) AS weighted_average
              FROM users u
              JOIN student s ON u.user_id = s.user_id
              JOIN grade_details gd ON s.student_id = gd.student_id
              JOIN course c ON gd.course_id = c.course_id
              WHERE u.username = sUserName
                  AND gd.course_id = cid
              GROUP BY gd.grade_type, gd.score, gd.weight
        )
        SELECT 
            assignment AS "Grade Type",
            score AS "Score",
            CASE 
                WHEN weighted_average >= 90 THEN 'A'
                WHEN weighted_average >= 80 THEN 'B'
                WHEN weighted_average >= 70 THEN 'C'
                WHEN weighted_average >= 60 THEN 'D'
                ELSE 'F'
            END AS "Letter Grade"
        FROM student_grades;
    END
    """
)

In [16]:
sUserName = 'stu_jhays64'
cid = 4

cursor.callproc('get_student_assignment_grades', (sUserName, cid,))

for result in cursor.stored_results():
    display_results(result)

,Grade Type,Score,Letter Grade
0,homework1,96,A
1,homework2,89,B
2,quiz,90,A
3,mid exam,100,A
4,final exam,93,A


In [17]:
cursor.close()
conn.close()